In [80]:
import os
import datetime
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import time
import logging

logger = logging.getLogger(__name__)

def localtime_to_str():
    return time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime())

logging.basicConfig(
    filename=f"data/output/gemini/log/gemini_{localtime_to_str()}.log", 
    encoding="utf-8",
    level=logging.DEBUG
    )

load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

def create_dir(base_dir="data/output/gemini/responses") -> str:
    """
    指定したベースディレクトリに、日付と連番を組み合わせた名称のディレクトリを作成する。
    既に同名のディレクトリが存在する場合、連番をインクリメントして新たなディレクトリを作成する。

    Args:
        base_dir: ベースとなるディレクトリのパス (デフォルト: "data/output/gemini/responses")

    Returns:
        str: 作成されたディレクトリの相対パス
    """

    today = datetime.date.today().strftime("%Y%m%d")
    num = 1

    while True:
        dir_name = f"{base_dir}/{today}_{num:03d}"  # 連番を3桁で表示
        try:
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
                return dir_name
            num += 1
        except OSError as e:
            print(f"ディレクトリ作成中にエラーが発生しました: {e}")

In [81]:
# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_schema": content.Schema(
    type = content.Type.OBJECT,
    enum = [],
    required = ["原因"],
    properties = {
      "原因": content.Schema(
        type = content.Type.ARRAY,
        items = content.Schema(
          type = content.Type.STRING,
        ),
      ),
    },
  ),
  "response_mime_type": "application/json",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
)

chat_session = model.start_chat(history=[])

In [82]:
import pandas as pd
import json
import prompts

In [83]:
global former_data
former_data = ""

In [84]:
def generate_json_response(
        data:pd.DataFrame, 
        response_name:str, 
        chat_session:genai.ChatSession,
        request_interval:int=10
        ) -> None:
    global former_data
    for i in tqdm(range(0, len(data))):
        if data.loc[i, "事故原因"] == former_data:
            logger.debug(f"Skipped: {data.values[i]}")
            data.loc[i, "事故原因"] = data.loc[i - 1, "事故原因"]
            continue
        cat = data.loc[i, "事故原因区分"]
        input_prompt = prompts.text[cat] + data.loc[i, "事故原因"]
        former_data = data.loc[i, "事故原因"]
        try:
            response = chat_session.send_message(input_prompt)
            temp = response.text
            res = json.loads(temp)
            logger.debug(temp)
        except Exception as e:
            logger.error("Error", e, temp)
            continue
        output_text = ""
        if isinstance(res[response_name], list):
            for text in res[response_name]:
                output_text += text + " "
        else:
            output_text = res[response_name]
        logger.debug(f"Output: {output_text}")
        data.loc[i, "事故原因"] = output_text
        time.sleep(request_interval)

In [85]:
import settings as st

output_dir = create_dir()

In [86]:
def processor(product:str)->None:
    data = pd.read_csv(f"data/output/gemini/prompt_data_250130/{product}_prompt.csv", encoding="utf-8-sig")
    generate_json_response(
        data=data, 
        response_name="原因", 
        chat_session=chat_session
        )
    print(data.head())
    data.to_csv(f"{output_dir}/{product}_res.csv", encoding="utf-8-sig", index=False)

In [87]:
processor(st.products_250115[5])

  0%|          | 0/53 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\chika\AppData\Local\Temp\ipykernel_24152\3555965812.py", line 17, in generate_json_response
    response = chat_session.send_message(input_prompt)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 578, in send_message
    response = self.model.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 830, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\ch

     No  被害の種類                  事故原因 事故原因区分
0  1043   2.重傷            紙詰まり 押し込み      E2
1  1640  11.火災     潤滑剤スプレー 可燃性ガス 火花      E2
2  2410   2.重傷               穴 挟み込み      E2
3  2487  11.火災  分解 エアースプレー 火花 可燃性ガス      E2
4  2693  11.火災      エアゾール缶 可燃性ガス 火花      E2


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\chika\AppData\Local\Temp\ipykernel_24152\3555965812.py", line 17, in generate_json_response
    response = chat_session.send_message(input_prompt)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 578, in send_message
    response = self.model.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 830, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\ch